# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, fbeta_score,make_scorer
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # convert to lowercase
    text = text.lower()
    
    # remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize
    token = nltk.tokenize.word_tokenize(text)
    
    # remove stop words
    token = [word for word in token if word not in nltk.corpus.stopwords.words('english')]
    
    # lemmatize
    token = [nltk.stem.wordnet.WordNetLemmatizer().lemmatize(word) for word in token]
    
    return token

In [4]:
quote = 'All that glitters is not gold. Hell is empty and all the devils are here. The lady doth protest too much, methinks.'

In [5]:
tokenize(quote)

['glitter',
 'gold',
 'hell',
 'empty',
 'devil',
 'lady',
 'doth',
 'protest',
 'much',
 'methinks']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multioutput', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=135)

In [8]:
# train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred_train = pipeline.predict(X_train)

In [10]:
for i in range(len(category_names)):
    cat = category_names[i]
    print(cat)
    print(classification_report(Y_train[cat], Y_pred_train[:, i]))

related
             precision    recall  f1-score   support

          0       0.98      0.97      0.98      4874
          1       0.99      0.99      0.99     15944
          2       0.96      0.95      0.95       154

avg / total       0.99      0.99      0.99     20972

request
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     17424
          1       0.99      0.93      0.96      3548

avg / total       0.99      0.99      0.99     20972

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     20876
          1       1.00      0.74      0.85        96

avg / total       1.00      1.00      1.00     20972

aid_related
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     12300
          1       0.99      0.97      0.98      8672

avg / total       0.99      0.98      0.98     20972

medical_help
             precision    recall  f1-sco

In [11]:
Y_pred_test = pipeline.predict(X_test)

In [12]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      1248
          1       0.84      0.91      0.87      3962
          2       0.28      0.47      0.35        34

avg / total       0.79      0.80      0.79      5244

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4318
          1       0.81      0.42      0.56       926

avg / total       0.87      0.88      0.86      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5222
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3056
          1       0.76      0.61      0.67      2188

avg / total       0.75      0.75      0.75      5244

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f24b333d7b8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multioutput',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_sta

In [14]:
parameters = {
    'multioutput__estimator__min_samples_leaf': [1, 2],
    'multioutput__estimator__min_samples_split': [2, 5],
}
cv = GridSearchCV(pipeline, param_grid=parameters)

In [15]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'multioutput__estimator__min_samples_leaf': [1, 2], 'multioutput__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
Y_pred_test = cv.predict(X_test)

In [17]:
for i in range(len(category_names)):
    cat = category_names[i]
    print(cat)
    print(classification_report(Y_test[cat], Y_pred_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.64      0.43      0.51      1248
          1       0.84      0.92      0.88      3962
          2       0.24      0.50      0.32        34

avg / total       0.79      0.80      0.79      5244

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4318
          1       0.81      0.44      0.57       926

avg / total       0.88      0.88      0.87      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5222
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3056
          1       0.74      0.60      0.66      2188

avg / total       0.75      0.75      0.74      5244

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multioutput', MultiOutputClassifier(AdaBoostClassifier()))
])

In [19]:
pipeline2.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [20]:
Y_pred_test = pipeline2.predict(X_test)

In [21]:
for i in range(len(category_names)):
    cat = category_names[i]
    print(cat)
    print(classification_report(Y_test[cat], Y_pred_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.60      0.09      0.15      1248
          1       0.77      0.98      0.86      3962
          2       0.50      0.12      0.19        34

avg / total       0.73      0.76      0.69      5244

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      4318
          1       0.76      0.51      0.61       926

avg / total       0.88      0.89      0.88      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5222
          1       0.12      0.05      0.07        22

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3056
          1       0.78      0.61      0.68      2188

avg / total       0.77      0.76      0.76      5244

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [22]:
import pickle
pkl_filename = 'classifier.pkl'
with open(pkl_filename, 'wb') as file:  
    pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.